# Analisi dei dati raccolti

## Setup

In [1]:
import pandas as pd

In [2]:
def print_data(series: pd.Series) -> None:
    print(series.value_counts())
    print()
    print(f"Massimo: {series.max()}\nMinimo: {series.min()}\nMedia: {series.mean()}\nVarianza: {series.var()}")

## Asconhashv12

In [3]:
path = "../asconhashv12/data/{}.csv"

### Armv7m

In [4]:
with open(path.format("armv7m"), "r") as f:
    ah_armv7m = pd.read_csv(f, delimiter=";")

#### PT vuoto

In [5]:
# Generazione hash
data = ah_armv7m["0B"]

print_data(data)

0B
86    918
87     79
85      3
Name: count, dtype: int64

Massimo: 87
Minimo: 85
Media: 86.076
Varianza: 0.07630030030030029


#### PT 8 byte

In [6]:
# Generazione hash
data = ah_armv7m["8B"]

print_data(data)

8B
106    876
107    121
105      3
Name: count, dtype: int64

Massimo: 107
Minimo: 105
Media: 106.118
Varianza: 0.11018618618618618


#### PT 16 byte

In [7]:
# Generazione hash
data = ah_armv7m["16B"]

print_data(data)

16B
125    839
127    121
126     40
Name: count, dtype: int64

Massimo: 127
Minimo: 125
Media: 125.282
Varianza: 0.444920920920921


#### PT 32 byte

In [8]:
# Generazione hash
data = ah_armv7m["32B"]

print_data(data)

32B
165    838
166    161
167      1
Name: count, dtype: int64

Massimo: 167
Minimo: 165
Media: 165.163
Varianza: 0.13856956956956956


#### PT 64 byte

In [9]:
# Generazione hash
data = ah_armv7m["64B"]

print_data(data)

64B
244    760
245    239
243      1
Name: count, dtype: int64

Massimo: 245
Minimo: 243
Media: 244.238
Varianza: 0.18353953953953953


#### PT 128 byte

In [10]:
# Generazione hash
data = ah_armv7m["128B"]

print_data(data)

128B
402    599
404    360
403     40
405      1
Name: count, dtype: int64

Massimo: 405
Minimo: 402
Media: 402.763
Varianza: 0.9077387387387388


#### PT 256 byte

In [11]:
# Generazione hash
data = ah_armv7m["256B"]

print_data(data)

256B
720    719
719    281
Name: count, dtype: int64

Massimo: 720
Minimo: 719
Media: 719.719
Varianza: 0.20224124124124118


#### PT 512 byte

In [12]:
# Generazione hash
data = ah_armv7m["512B"]

print_data(data)

512B
1354    518
1356    360
1355    122
Name: count, dtype: int64

Massimo: 1356
Minimo: 1354
Media: 1354.842
Varianza: 0.8538898898898897


#### PT 1024 byte

In [13]:
# Generazione hash
data = ah_armv7m["1024B"]

print_data(data)

1024B
2624    640
2623    358
2626      1
2625      1
Name: count, dtype: int64

Massimo: 2626
Minimo: 2623
Media: 2623.645
Varianza: 0.23721221221221211


### Armv7m_lowsize

In [14]:
with open(path.format("armv7m_lowsize"), "r") as f:
    ah_lowsize = pd.read_csv(f, delimiter=";")

#### PT vuoto

In [15]:
# Generazione hash
data = ah_lowsize["0B"]

print_data(data)

0B
70    918
71     81
72      1
Name: count, dtype: int64

Massimo: 72
Minimo: 70
Media: 70.083
Varianza: 0.07818918918918918


#### PT 8 byte

In [16]:
# Generazione hash
data = ah_lowsize["8B"]

print_data(data)

8B
87    880
88    120
Name: count, dtype: int64

Massimo: 88
Minimo: 87
Media: 87.12
Varianza: 0.1057057057057057


#### PT 16 byte

In [17]:
# Generazione hash
data = ah_lowsize["16B"]

print_data(data)

16B
103    840
105    121
104     39
Name: count, dtype: int64

Massimo: 105
Minimo: 103
Media: 103.281
Varianza: 0.4444834834834834


#### PT 32 byte

In [18]:
# Generazione hash
data = ah_lowsize["32B"]

print_data(data)

32B
136    479
137    361
138    160
Name: count, dtype: int64

Massimo: 138
Minimo: 136
Media: 136.681
Varianza: 0.5377767767767768


#### PT 64 byte

In [19]:
# Generazione hash
data = ah_lowsize["64B"]

print_data(data)

64B
203    800
204    198
202      2
Name: count, dtype: int64

Massimo: 204
Minimo: 202
Media: 203.196
Varianza: 0.16174574574574574


#### PT 128 byte

In [20]:
# Generazione hash
data = ah_lowsize["128B"]

print_data(data)

128B
335    637
336    361
338      1
337      1
Name: count, dtype: int64

Massimo: 338
Minimo: 335
Media: 335.366
Varianza: 0.2402842842842843


#### PT 256 byte

In [21]:
# Generazione hash
data = ah_lowsize["256B"]

print_data(data)

256B
601    600
600    399
602      1
Name: count, dtype: int64

Massimo: 602
Minimo: 600
Media: 600.602
Varianza: 0.24183783783783785


#### PT 512 byte

In [22]:
# Generazione hash
data = ah_lowsize["512B"]

print_data(data)

512B
1132    840
1133    159
1131      1
Name: count, dtype: int64

Massimo: 1133
Minimo: 1131
Media: 1132.158
Varianza: 0.13517117117117117


#### PT 1024 byte

In [23]:
# Generazione hash
data = ah_lowsize["1024B"]

print_data(data)

1024B
2193    798
2194    201
2195      1
Name: count, dtype: int64

Massimo: 2195
Minimo: 2193
Media: 2193.203
Varianza: 0.1639549549549549


### Armv7m_small

In [24]:
with open(path.format("armv7m_small"), "r") as f:
    ah_small = pd.read_csv(f, delimiter=";")

#### PT vuoto

In [25]:
# Generazione hash
data = ah_small["0B"]

print_data(data)

0B
70    917
71     43
72     40
Name: count, dtype: int64

Massimo: 72
Minimo: 70
Media: 70.123
Varianza: 0.18805905905905906


#### PT 8 byte

In [26]:
# Generazione hash
data = ah_small["8B"]

print_data(data)

8B
87    880
88    120
Name: count, dtype: int64

Massimo: 88
Minimo: 87
Media: 87.12
Varianza: 0.1057057057057057


#### PT 16 byte

In [27]:
# Generazione hash
data = ah_small["16B"]

print_data(data)

16B
103    839
105     81
104     80
Name: count, dtype: int64

Massimo: 105
Minimo: 103
Media: 103.242
Varianza: 0.3457817817817818


#### PT 32 byte

In [28]:
# Generazione hash
data = ah_small["32B"]

print_data(data)

32B
136    758
138    160
137     82
Name: count, dtype: int64

Massimo: 138
Minimo: 136
Media: 136.402
Varianza: 0.5609569569569569


#### PT 64 byte

In [29]:
# Generazione hash
data = ah_small["64B"]

print_data(data)

64B
203    760
204    198
202     42
Name: count, dtype: int64

Massimo: 204
Minimo: 202
Media: 203.156
Varianza: 0.21587987987987983


#### PT 128 byte

In [30]:
# Generazione hash
data = ah_small["128B"]

print_data(data)

128B
335    637
336    361
338      1
337      1
Name: count, dtype: int64

Massimo: 338
Minimo: 335
Media: 335.366
Varianza: 0.2402842842842843


#### PT 256 byte

In [31]:
# Generazione hash
data = ah_small["256B"]

print_data(data)

256B
601    600
600    399
602      1
Name: count, dtype: int64

Massimo: 602
Minimo: 600
Media: 600.602
Varianza: 0.24183783783783785


#### PT 512 byte

In [32]:
# Generazione hash
data = ah_small["512B"]

print_data(data)

512B
1132    801
1133    159
1131     40
Name: count, dtype: int64

Massimo: 1133
Minimo: 1131
Media: 1132.119
Varianza: 0.18502402402402401


#### PT 1024 byte

In [33]:
# Generazione hash
data = ah_small["1024B"]

print_data(data)

1024B
2193    798
2194    201
2195      1
Name: count, dtype: int64

Massimo: 2195
Minimo: 2193
Media: 2193.203
Varianza: 0.1639549549549549


### Bi32_armv7m

In [34]:
with open(path.format("bi32_armv7m"), "r") as f:
    ah_bi32 = pd.read_csv(f, delimiter=";")

#### PT vuoto

In [35]:
# Generazione hash
data = ah_bi32["0B"]

print_data(data)

0B
71    920
72     80
Name: count, dtype: int64

Massimo: 72
Minimo: 71
Media: 71.08
Varianza: 0.0736736736736737


#### PT 8 byte

In [36]:
# Generazione hash
data = ah_bi32["8B"]

print_data(data)

8B
86    918
87     82
Name: count, dtype: int64

Massimo: 87
Minimo: 86
Media: 86.082
Varianza: 0.07535135135135136


#### PT 16 byte

In [37]:
# Generazione hash
data = ah_bi32["16B"]

print_data(data)

16B
102    878
103    122
Name: count, dtype: int64

Massimo: 103
Minimo: 102
Media: 102.122
Varianza: 0.10722322322322322


#### PT 32 byte

In [38]:
# Generazione hash
data = ah_bi32["32B"]

print_data(data)

32B
132    881
133    119
Name: count, dtype: int64

Massimo: 133
Minimo: 132
Media: 132.119
Varianza: 0.10494394394394395


#### PT 64 byte

In [39]:
# Generazione hash
data = ah_bi32["64B"]

print_data(data)

64B
193    799
194    201
Name: count, dtype: int64

Massimo: 194
Minimo: 193
Media: 193.201
Varianza: 0.16075975975975976


#### PT 128 byte

In [40]:
# Generazione hash
data = ah_bi32["128B"]

print_data(data)

128B
315    681
316    319
Name: count, dtype: int64

Massimo: 316
Minimo: 315
Media: 315.319
Varianza: 0.21745645645645648


#### PT 256 byte

In [41]:
# Generazione hash
data = ah_bi32["256B"]

print_data(data)

256B
560    557
559    440
562      2
561      1
Name: count, dtype: int64

Massimo: 562
Minimo: 559
Media: 559.565
Varianza: 0.260035035035035


#### PT 512 byte

In [42]:
# Generazione hash
data = ah_bi32["512B"]

print_data(data)

512B
1050    679
1049    321
Name: count, dtype: int64

Massimo: 1050
Minimo: 1049
Media: 1049.679
Varianza: 0.2181771771771771


#### PT 1024 byte

In [43]:
# Generazione hash
data = ah_bi32["1024B"]

print_data(data)

1024B
2028    958
2029     40
2027      2
Name: count, dtype: int64

Massimo: 2029
Minimo: 2027
Media: 2028.038
Varianza: 0.040596596596596576


### Ref

In [44]:
with open(path.format("ref"), "r") as f:
    ah_ref = pd.read_csv(f, delimiter=";")

#### PT vuoto

In [45]:
# Generazione hash
data = ah_ref["0B"]

print_data(data)

0B
300    680
301    320
Name: count, dtype: int64

Massimo: 301
Minimo: 300
Media: 300.32
Varianza: 0.2178178178178179


#### PT 8 byte

In [46]:
# Generazione hash
data = ah_ref["8B"]

print_data(data)

8B
361    638
362    360
363      2
Name: count, dtype: int64

Massimo: 363
Minimo: 361
Media: 361.364
Varianza: 0.2357397397397397


#### PT 16 byte

In [47]:
# Generazione hash
data = ah_ref["16B"]

print_data(data)

16B
422    480
424    398
423    122
Name: count, dtype: int64

Massimo: 424
Minimo: 422
Media: 422.918
Varianza: 0.872148148148148


#### PT 32 byte

In [48]:
# Generazione hash
data = ah_ref["32B"]

print_data(data)

32B
546    520
545    478
547      2
Name: count, dtype: int64

Massimo: 547
Minimo: 545
Media: 545.524
Varianza: 0.25367767767767774


#### PT 64 byte

In [49]:
# Generazione hash
data = ah_ref["64B"]

print_data(data)

64B
791    758
790    239
792      3
Name: count, dtype: int64

Massimo: 792
Minimo: 790
Media: 790.764
Varianza: 0.18649049049049052


#### PT 128 byte

In [50]:
# Generazione hash
data = ah_ref["128B"]

print_data(data)

128B
1282    718
1283    279
1281      3
Name: count, dtype: int64

Massimo: 1283
Minimo: 1281
Media: 1282.276
Varianza: 0.20603003003002998


#### PT 256 byte

In [51]:
# Generazione hash
data = ah_ref["256B"]

print_data(data)

256B
2263    758
2264    241
2265      1
Name: count, dtype: int64

Massimo: 2265
Minimo: 2263
Media: 2263.243
Varianza: 0.18613713713713717


#### PT 512 byte

In [52]:
# Generazione hash
data = ah_ref["512B"]

print_data(data)

512B
4226    799
4227    200
4229      1
Name: count, dtype: int64

Massimo: 4229
Minimo: 4226
Media: 4226.203
Varianza: 0.16795895895895896


#### PT 1024 byte

In [53]:
# Generazione hash
data = ah_ref["1024B"]

print_data(data)

1024B
8151    877
8152    121
8153      2
Name: count, dtype: int64

Massimo: 8153
Minimo: 8151
Media: 8151.125
Varianza: 0.11348848848848848
